We first implement the procedures to compute the quotient, remainder, and highest common factor of polynomials with coefficients in a finite field $GF(p)$, and also an efficient method for modular exponentiation.Let $A(x)$ be the dividend polynomial and $B(x)$ be the divisor polynomial of degree $m$. Let $p$ be some prime modulus.

1. This procedure implements polynomial long division over a finite field $GF(p)$. All arithmetic operations on the coefficients are performed modulo $p$.

    *   Initialise $Q(x) = 0$ and $R(x) = A(x)$.
    *   While $\deg(R) \geq \deg(B)$:
        *   Calculate the inverse of the leading coefficient of $B(x)$ modulo $p$.
        *   Determine the leading term of $R(x)$, which is $lc_R x^k$, and the leading term of $B(x)$, which is $lc_B x^m$.
        *   Create a term $T(x) = (lc_R lc_B^{-1} \bmod p) x^{k-m}$ and add this to the quotient $Q(x) = Q(x) + T(x)$.
        *   Update the remainder $R(x) = R(x) - T(x)B(x)$, modulo $p$.
    *   Return $Q(x)$ and $R(x)$.

2. The highest common factor of two polynomials is found using the Euclidean algorithm, which repeatedly uses the division procedure above.

    *   Let $R_0 = A(x)$ and $R_1 = B(x)$.
    *   While $R_1(x) \neq 0$:
        *   Use the polynomial division procedure to find the remainder of $R_0(x)$ divided by $R_1(x)$ which is denoted $R_2(x)$.
        *   Update the polynomials $R_0(x) = R_1(x)$ and $R_1(x) = R_2(x)$.
    *   The result $\mathop{HCF}(A(x), B(x))$ is the last non-zero remainder, $R_0(x)$. Make the HCF monic by dividing it by its leading coefficient modulo $p$.

3. To efficiently compute $P(x)^k \mod{M(x)}$ for a large integer $k$, the method of binary exponentiation is used. This is significantly faster than performing $k-1$ successive multiplications as the number of polynomial multiplications required is proportional to $\log(k)$.

    *   If $k = 0$, then return $1$.
    *   Initialise C = 1.
    *   Define the base as $B(x) = P(x) \mod{M(x)}$.
    *   While $k > 0$:
        *   If $k$ is odd, then update $C = (CB(x)) \mod{M(x)}$.
        *   Square the base $B(x) = B(x)^2 \mod{M(x)}$.
        *   Halve the exponent $k = \lfloor k / 2 \rfloor$.
    *   Return $C$.

In [13]:
import numbers

class Polynomial:
    '''
    A class to represent polynomials with coefficients in a finite field GF(p).
    All coefficient arithmetic is performed modulo p.
    '''

    def __init__(self, coefficients, p):
        '''
        Initializes a polynomial.

        Args:
            coefficients: A list of coefficients in descending degree order
                or a dict mapping degree to coefficient.
            p: The prime modulus for the finite field GF(p).
        '''
        if not isinstance(p, int) or p <= 1:
            raise ValueError("p must be a prime integer greater than 1.")
        self.p = p

        if isinstance(coefficients, list):
            # Convert list [a_n, ..., a_0] to dict {n: a_n, ...}
            self.coeffs = {len(coefficients) - 1 - i: c % self.p
                           for i, c in enumerate(coefficients) if c % self.p != 0}
        elif isinstance(coefficients, dict):
            self.coeffs = {deg: c % self.p for deg, c in coefficients.items() if c % self.p != 0}
        else:
            raise TypeError("Coefficients must be a list or a dictionary.")

        # Handle the zero polynomial
        if not self.coeffs:
            self.coeffs = {0: 0}

    def degree(self):
        '''
        Returns the degree of the polynomial.
        '''
        if self.coeffs == {0: 0}:
            return -1
        return max(self.coeffs.keys())

    def __str__(self):
        '''
        Returns a string representation of the polynomial.
        '''
        if self.degree() == -1:
            return "0"

        parts = []
        for deg in sorted(self.coeffs.keys(), reverse=True):
            coeff = self.coeffs[deg]
            if coeff == 1 and deg != 0:
                coeff_str = ""
            else:
                coeff_str = str(coeff)
            if deg > 1:
                var_str = f"x^{deg}"
            elif deg == 1:
                var_str = "x"
            else:
                var_str = ""
            if deg != 0 and coeff != 1:
                 parts.append(f"{coeff_str}{var_str}")
            elif deg != 0 and coeff == 1:
                 parts.append(var_str)
            else:
                 parts.append(coeff_str)
        return " + ".join(parts)

    def __repr__(self):
        return f"Polynomial({self.coeffs}, p={self.p})"

    def __eq__(self, other):
        '''
        Checks for polynomial equality.
        '''
        return self.coeffs == other.coeffs and self.p == other.p

    def __add__(self, other):
        '''
        Polynomial addition.
        '''
        new_coeffs = self.coeffs.copy()
        for deg, coeff in other.coeffs.items():
            new_coeffs[deg] = (new_coeffs.get(deg, 0) + coeff) % self.p
        return Polynomial(new_coeffs, self.p)

    def __sub__(self, other):
        '''
        Polynomial subtraction.
        '''
        new_coeffs = self.coeffs.copy()
        for deg, coeff in other.coeffs.items():
            new_coeffs[deg] = (new_coeffs.get(deg, 0) - coeff) % self.p
        return Polynomial(new_coeffs, self.p)

    def __mul__(self, other):
        '''
        Polynomial multiplication.
        '''
        if isinstance(other, numbers.Number): # Scalar multiplication
            new_coeffs = {deg: (c * other) % self.p for deg, c in self.coeffs.items()}
            return Polynomial(new_coeffs, self.p)
        new_coeffs = {}
        for deg1, coeff1 in self.coeffs.items():
            for deg2, coeff2 in other.coeffs.items():
                new_deg = deg1 + deg2
                new_coeff = (coeff1 * coeff2) % self.p
                new_coeffs[new_deg] = (new_coeffs.get(new_deg, 0) + new_coeff) % self.p
        return Polynomial(new_coeffs, self.p)

    def __divmod__(self, divisor):
        '''
        Polynomial long division to find quotient and remainder.
        '''
        if divisor.degree() == -1:
            raise ZeroDivisionError("Cannot divide by the zero polynomial.")

        p = self.p
        remainder = Polynomial(self.coeffs.copy(), p)
        quotient = Polynomial({}, p)

        divisor_deg = divisor.degree()
        divisor_lc = divisor.coeffs[divisor_deg]

        # Modular inverse of the divisor's leading coefficient
        inv_lc = pow(divisor_lc, -1, p)

        while remainder.degree() >= divisor_deg:
            rem_deg = remainder.degree()
            rem_lc = remainder.coeffs[rem_deg]

            term_deg = rem_deg - divisor_deg
            term_coeff = (rem_lc * inv_lc) % p

            quotient.coeffs[term_deg] = term_coeff

            term_poly = Polynomial({term_deg: term_coeff}, p)
            remainder = remainder - (term_poly * divisor)

        return quotient, remainder

    def __floordiv__(self, other):
        '''
        Returns the quotient of polynomial division.
        '''
        return divmod(self, other)[0]

    def __mod__(self, other):
        '''
        Returns the remainder of polynomial division.
        '''
        return divmod(self, other)[1]

In [14]:
def polynomial_hcf(a, b):
    '''
    Computes the highest common factor (HCF) of two polynomials over GF(p)
    using the Euclidean Algorithm.
    '''
    if a.p != b.p:
        raise ValueError("Polynomials must be in the same field GF(p).")
    while b.degree() != -1:
        a, b = b, a % b
    # Make the HCF monic
    if a.degree() != -1:
        lc = a.coeffs[a.degree()]
        inv_lc = pow(lc, -1, a.p)
        a = a * inv_lc
    return a

def polynomial_power(base, exp, modulus):
    '''
    Computes (base^exp) % modulus for polynomials efficiently
    using the method of binary exponentiation (exponentiation by squaring).
    '''
    if base.p != modulus.p:
        raise ValueError("Polynomials must be in the same field GF(p).")
    p = base.p
    result = Polynomial([1], p) # Multiplicative identity
    base %= modulus
    while exp > 0:
        # If exponent is odd, multiply result with base
        if exp % 2 == 1:
            result = (result * base) % modulus
        # Exponent is now even, base can be squared
        exp //= 2
        base = (base * base) % modulus
    return result

In [15]:
p = 5  # We will work in the finite field GF(5)

print("--- Question 1: Polynomial Procedures over GF(p) ---")
print(f"All calculations will be performed in the finite field GF({p}).\n")

print("Quotient and remainder computation")
A = Polynomial([3, 1, 2, 4, 1], p)  # 3x^4 + x^3 + 2x^2 + 4x + 1
B = Polynomial([2, 3, 1], p)      # 2x^2 + 3x + 1
print(f"  Dividend A(x)  = {A}")
print(f"  Divisor B(x)   = {B}")
quotient, remainder = divmod(A, B)
print(f"  Quotient Q(x)  = {quotient}") # Expected: 4x^2 + 2x + 1
print(f"  Remainder R(x) = {remainder}")   # Expected: 4x

print("\nHighest common factor computation")
C = Polynomial([1, 1, 1, 1], p) # x^3 + x^2 + x + 1
D = Polynomial([1, 0, 4], p)   # x^2 + 4
print(f"  Polynomial C(x) = {C}")
print(f"  Polynomial D(x) = {D}")
hcf = polynomial_hcf(C, D)
print(f"  HCF(C, D) = {hcf}") # Expected: x + 1

print("\nEfficient computation of a large power")
P = Polynomial([1, 1], p)         # P(x) = x + 1
M = Polynomial([1, 0, 1], p)      # M(x) = x^2 + 1
k = 100                           # A large power
print(f"  Base Polynomial P(x) = {P}")
print(f"  Modulus Polynomial M(x) = {M}")
print(f"  Exponent k = {k}")
result_poly = polynomial_power(P, k, M)
print(f"  Result of (P(x)^k) mod M(x) is: {result_poly}")

--- Question 1: Polynomial Procedures over GF(p) ---
All calculations will be performed in the finite field GF(5).

Quotient and remainder computation
  Dividend A(x)  = 3x^4 + x^3 + 2x^2 + 4x + 1
  Divisor B(x)   = 2x^2 + 3x + 1
  Quotient Q(x)  = 4x^2 + 2x + 1
  Remainder R(x) = 4x

Highest common factor computation
  Polynomial C(x) = x^3 + x^2 + x + 1
  Polynomial D(x) = x^2 + 4
  HCF(C, D) = x + 1

Efficient computation of a large power
  Base Polynomial P(x) = x + 1
  Modulus Polynomial M(x) = x^2 + 1
  Exponent k = 100
  Result of (P(x)^k) mod M(x) is: 1
